In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import numpy as np

# ---------------------------
# DIP Preprocessing Function (fixed)
# ---------------------------
def dip_preprocess(image):
    image = image.numpy()  # convert Tensor -> numpy

    # Convert to uint8 safely
    if image.dtype != np.uint8:
        if image.max() <= 1.0:
            image = (image * 255).astype(np.uint8)
        else:
            image = image.astype(np.uint8)

    # Ensure it’s 3-channel RGB
    if image.ndim == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.shape[2] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)

    # Convert to grayscale (uint8)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Histogram equalization (safe since gray is uint8)
    eq = cv2.equalizeHist(gray)

    # Canny edge detection
    edges = cv2.Canny(eq, 100, 200)

    # Overlay edges on equalized image
    overlay = cv2.addWeighted(eq, 0.8, edges, 0.2, 0)

    # Convert back to 3-channel RGB
    overlay = cv2.merge([overlay, overlay, overlay])

    # Normalize to [0,1]
    overlay = overlay.astype(np.float32) / 255.0
    return overlay


# ---------------------------
# Batch-safe preprocessing
# ---------------------------
def preprocess_dataset(dataset):
    def _map_fn(x, y):
        x = tf.map_fn(
            lambda img: tf.py_function(func=dip_preprocess, inp=[img], Tout=tf.float32),
            x,
            dtype=tf.float32
        )
        x.set_shape((None, 48, 48, 3))
        y = tf.cast(y, tf.int32)
        return x, y

    return dataset.map(_map_fn, num_parallel_calls=tf.data.AUTOTUNE)


# ---------------------------
# Load Datasets
# ---------------------------
train_ds = tf.keras.utils.image_dataset_from_directory(
    "../images/train",
    color_mode="rgb",
    image_size=(48, 48),
    batch_size=32,
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "../images/validation",
    color_mode="rgb",
    image_size=(48, 48),
    batch_size=32
)

class_names = train_ds.class_names
print("Detected classes:", class_names)

# Apply DIP preprocessing
train_ds = preprocess_dataset(train_ds)
val_ds = preprocess_dataset(val_ds)

train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

# ---------------------------
# CNN Model
# ---------------------------
model = keras.Sequential([
    layers.Rescaling(1./1, input_shape=(48, 48, 3)),

    layers.Conv2D(512, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.3),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    layers.Dense(len(class_names), activation='softmax')
])

# ---------------------------
# Compile and Train
# ---------------------------
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
)

test_loss, test_acc = model.evaluate(val_ds)
print(f"✅ Validation accuracy: {test_acc * 100:.2f}%")


Found 28822 files belonging to 7 classes.
Found 7066 files belonging to 7 classes.
Detected classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Epoch 1/50


2025-11-08 02:10:54.773879: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-08 02:10:57.038696: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3156', 520 bytes spill stores, 520 bytes spill loads



901/901 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.2605 - loss: 2.1243

2025-11-08 02:13:01.259455: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.21GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-11-08 02:13:16.047645: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.19GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


901/901 ━━━━━━━━━━━━━━━━━━━━ 146s 147ms/step - accuracy: 0.3201 - loss: 1.8496 - val_accuracy: 0.4108 - val_loss: 1.5272
Epoch 2/50
901/901 ━━━━━━━━━━━━━━━━━━━━ 120s 133ms/step - accuracy: 0.4515 - loss: 1.4297 - val_accuracy: 0.4742 - val_loss: 1.3781
Epoch 3/50
901/901 ━━━━━━━━━━━━━━━━━━━━ 128s 141ms/step - accuracy: 0.4973 - loss: 1.3224 - val_accuracy: 0.4060 - val_loss: 1.5791
Epoch 4/50
901/901 ━━━━━━━━━━━━━━━━━━━━ 129s 142ms/step - accuracy: 0.5175 - loss: 1.2690 - val_accuracy: 0.5207 - val_loss: 1.2581
Epoch 5/50
901/901 ━━━━━━━━━━━━━━━━━━━━ 131s 144ms/step - accuracy: 0.5353 - loss: 1.2258 - val_accuracy: 0.5306 - val_loss: 1.2341
Epoch 6/50
901/901 ━━━━━━━━━━━━━━━━━━━━ 128s 140ms/step - accuracy: 0.5615 - loss: 1.1707 - val_accuracy: 0.5395 - val_loss: 1.2144
Epoch 7/50
901/901 ━━━━━━━━━━━━━━━━━━━━ 128s 141ms/step - accuracy: 0.5767 - loss: 1.1344 - val_accuracy: 0.5303 - val_loss: 1.2320
Epoch 8/50
901/901 ━━━━━━━━━━━━━━━━━━━━ 128s 141ms/step - accuracy: 0.5903 - loss: 1.08